# Data Science Incubator Project 5

## Imports and settings

In [1]:
# Python imports
import random as rn
import pickle

# Numerical imports
import numpy as np
import pandas as pd

# NLP packages
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Plotting imports
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting settings
sns.set()
sns.set_palette("colorblind")
sns.set_style("ticks")

# Seed randomness
rn.seed(0)

[nltk_data] Downloading package stopwords to /home/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load data - and split (optional)

In [2]:
def split_data(sample_rate=1e-3) -> None:
    for split in ["test", "train"]:
        data = pd.read_csv(f"./data/{split}.csv", header=None, names=["rating", "title", "review"])
        data = data.sample(frac=sample_rate)
        data.to_csv(f"./data/{split}_sample.csv", index=False)

# split_data(sample_rate=1e-2)  # 1% of data
train = pd.read_csv("./data/train_sample.csv")
test = pd.read_csv("./data/test_sample.csv")

display(train)
display(test)

,rating,title,review
0,5,Fur immer,This double dvd by Germany's rock goddess is t...
1,4,Decent beginner kit,I like this kit for a beginner's kit in roboti...
2,5,thad lones mel lewis,everything by thad and mel is great. am still ...
3,3,Not as comfy as I expected,"When I received them, thought that I will be w..."
4,3,Wedding music CD,The versions were okay overall. About 3 of the...
...,...,...,...
29995,1,More for the children (Ages 1-8) ...,This animated movie is not at all as great as ...
29996,1,damage immediately,One of the extensions ceases to function after...
29997,4,Box Making,The book gave me several new ideas for design ...
29998,5,Can't Get Enough,I CANNOT GET ENOUGH OF JANE GREEN!!! Loved it ...


,rating,title,review
0,5,GREAT MUSIC FOR EVERYONE !!!!,In this cd it contains GREATNESS!!! TRUST ME T...
1,4,Quite pleased,"Great sound, easy installation. Only negative ..."
2,5,Great movie,"Had this movie forever on dvd, we figured it w..."
3,3,PUDDLES WATER,"IF YOU, PUT THIS DIEVICE ON LOW OUTPUT IT WILL..."
4,2,Horrible tech support,I can't comment on the quality of the game bec...
...,...,...,...
6495,2,poor sound quality,"I received this phone,it is cute and retro hop..."
6496,2,Doesn't seem very accurate,This thing takes forever to get up to temperat...
6497,2,leaks,I do not like these bags. The twist tie closur...
6498,4,Good Stuff,This nailer works pretty good but at least wit...


## Preprocess data

In [22]:

train = train[0:5]

def build_corpus():
    ps = PorterStemmer()
    corpus = []

    for index in range(len(train)):
        title = train.title[index]
        review = train.review[index]

        title = re.sub(r"[^a-zA-Z]", " ", title).lower().split()
        review = re.sub(r"[^a-zA-Z]", " ", review).lower().split()

        title = [ps.stem(word) for word in title if not word in set(stopwords.words("english"))]
        review = [ps.stem(word) for word in review if not word in set(stopwords.words("english"))]

        corpus.append(title + review)
        
    with open("./data/corpus.pck", "wb+") as fout:
        pickle.dump(corpus, fout)
        
def load_corpus():
    with open("./data/corpus.pck", "rb") as fin:
        return pickle.load(fin)

In [23]:
build_corpus()

In [ ]:
load_corpus()

In [ ]:
# build gensim model
# construct X_train and Y_train
    # making average vector of review + title
# train model

# # --------------------------------------------------
# classifier = GaussianNB()
# classifier.fit(X_train, y_train)

# # Predicting the Test set results
# y_pred = classifier.predict(X_test)
# # --------------------------------------------------

# evaluate - f1 scores, precision, recall
# evaluate - random guessing
# graph results
# graph histogram of review scores of truth vs predictions